In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

### Hyperparameter: 모델 최적화 과정을 제어할 수 있는 조절 가능한 매개변수
- epoch: 데이터셋을 반복하는 횟수
- batch size: 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플의 수
- learning rate: 값이 작을수록 학습 속도가 느려지고, 값이 크면 학습 중 예측할 수 없는 동작이 발생할 수 있음

In [2]:
lr = 1e-3
batch_size = 64
epochs = 5

### Optimization Loop
- train loop: 학습용 데이터셋을 반복하고 최적의 매개변수로 수렴
- validation, test loop: 모델 성능이 개선되고 있는지를 확인하기 위해 테스트 데이터셋을 반복함

### Loss function
- 획득한 결과와 실제 값 사이의 틀린 정도를 측정, 학습 중에 이 값을 최소화하는 방향으로 진행, 예측과 label을 비교하여 loss 계산
    + MSE: regression task에 사용
    + NLL (Negative Log Likelihood): classification에 사용
    + CrossEntropyLoss (LogSoftmax와 NLL 합침)

In [4]:
loss_fn = nn.CrossEntropyLoss()

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    
    for batch, (X,y) in enumerate(dataloader):
        # calculate prediction and loss
        pred =model(X)
        loss = loss_fn(pred, y)
        
        # backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"loss: {loss:>.7f} [{current:>5d}/{size:>5d}")
            
            
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches =len(dataloader)
    
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
            
    
    
    
    

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

epochs = 10

for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("done")

Epoch 1
-------------------
loss: 2.3058732 [    0/60000
loss: 2.2969346 [ 6400/60000
loss: 2.2736914 [12800/60000
loss: 2.2686176 [19200/60000
loss: 2.2467835 [25600/60000
loss: 2.2228527 [32000/60000
loss: 2.2221441 [38400/60000
loss: 2.1955137 [44800/60000
loss: 2.2006226 [51200/60000
loss: 2.1602459 [57600/60000
Test Error: 
 Accuracy: 46.0%, Avg loss: 2.158386 

Epoch 2
-------------------
loss: 2.1719401 [    0/60000
loss: 2.1615202 [ 6400/60000
loss: 2.1090696 [12800/60000
loss: 2.1241095 [19200/60000
loss: 2.0640786 [25600/60000
loss: 2.0159433 [32000/60000
loss: 2.0339916 [38400/60000
loss: 1.9647261 [44800/60000
loss: 1.9733983 [51200/60000
loss: 1.8974938 [57600/60000
Test Error: 
 Accuracy: 57.2%, Avg loss: 1.896677 

Epoch 3
-------------------
loss: 1.9298425 [    0/60000
loss: 1.8981118 [ 6400/60000
loss: 1.7915467 [12800/60000
loss: 1.8311719 [19200/60000
loss: 1.7081628 [25600/60000
loss: 1.6745000 [32000/60000
loss: 1.6891218 [38400/60000
loss: 1.5961089 [44800/60000
